In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
cpu_count = os.cpu_count()
print(f"Number of CPUs: {cpu_count}")

Number of CPUs: 12


In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA L4 (UUID: GPU-5bb87bec-c975-ac23-8bda-76f25026445e)


In [ ]:
import torch
gpu_count = torch.cuda.device_count()
print(f"Number of GPUs: {gpu_count}")

Number of GPUs: 1


In [ ]:
cd /content/drive/MyDrive/Kaggle/ttn

/content/drive/MyDrive/Kaggle/ttn


# import

In [ ]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# 自定义模块
from Func import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

# EDA

## MISSING & CLEANING

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('data2/train.csv')

In [ ]:
test = pd.read_csv('data2/test.csv')

In [ ]:
pd.set_option('max_colwidth',200)

ttn.head(5)

In [ ]:
def missing (df):
    """
    计算每一列的缺失值及占比
    """
    missing_number = df.isnull().sum().sort_values(ascending=False)              # 每一列的缺失值求和后降序排序
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)          # 每一列缺失值占比
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])      # 合并为一个DataFrame
    return missing_values

In [ ]:
missing(ttn)

In [ ]:
def find_index(data_col, val):
    """
    查询某值在某列中第一次出现位置的索引，没有则返回-1

    :param data_col: 查询的列
    :param val: 具体取值
    """
    val_list = [val]
    if data_col.isin(val_list).sum() == 0:
        index = -1
    else:
        index = data_col.isin(val_list).idxmax()
    return index

In [ ]:
for col in numeric_cols:
    print(find_index(ttn[col], ' '))

In [ ]:
ttn[category_cols].nunique()

## save csv

In [ ]:
ttn.to_csv('ttn.csv', index=False)

In [ ]:
ttn = pd.read_csv('ttn.csv')

# openfe

In [ ]:
from openfe import OpenFE, transform

In [ ]:
ofe = OpenFE()
features = ofe.fit(data=train.drop('Survived', axis=1),
                   #min_candidate_features=2000,
                   #verbose=False,
                   #n_data_blocks=8,
                   label=train['Survived'],
                   feature_boosting=False,
                   n_jobs=10)
# , task='regression', metric='rmse' **verbose=0** verbose=False



In [ ]:
features

In [ ]:
train_x, test = transform(train.drop('Survived', axis=1), test, features, n_jobs=10)

train_x.reset_index(drop=True, inplace=True)

train = pd.concat([train_x, train['Survived']], axis=1)

In [ ]:
train.to_csv('openfe_train.csv', index=False)

In [ ]:
test.to_csv('openfe_test.csv', index=False)

In [ ]:
train = pd.read_csv('openfe_train.csv')

In [ ]:
test = pd.read_csv('openfe_test.csv')

# autogluon

## install

In [ ]:
!pip install openfe

In [ ]:
!python -m pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 29.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requireme

In [ ]:
!pip install dask[dataframe]

In [ ]:
!pip install --upgrade datasets

!pip show pyarrow
!pip show datasets
!pip uninstall -y pyarrow
!pip install pyarrow --no-cache-dir
!pip install pyarrow
!pip install transformers
!pip install --upgrade datasets
!pip install datasets
! pip install pyarrow
! pip install pyarrow.parquet
!pip install transformers

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from autogluon.tabular import TabularDataset, TabularPredictor

## featrue generator

In [ ]:
# 划分训练集和测试集
train, test = train_test_split(train, random_state=22, test_size=0.1)

In [ ]:
feature_generator = AutoMLPipelineFeatureGenerator()

label = 'Survived'

X_train = train.drop(labels=[label], axis=1)
y_train = train[label]

X_train_transformed = feature_generator.fit_transform(X=X_train, y=y_train)

In [ ]:
X_test_transformed = feature_generator.transform(test)

In [ ]:
len(X_val_transformed.columns), X_val_transformed.columns

In [ ]:
X_train_transformed.to_csv('featuresCreation/gluon_train.csv', index=False)
X_test_transformed.to_csv('featuresCreation/gluon_test.csv', index=False)

In [ ]:
X_gluon_train = pd.read_csv('featuresCreation/gluon_train.csv')

In [ ]:
X_gluon_test = pd.read_csv('featuresCreation/gluon_test.csv')

## utils

In [ ]:
import shutil
import os

# Specify the path to the .cache directory
cache_dir = os.path.expanduser('~/.cache')

# Check if the directory exists
if os.path.exists(cache_dir):
    # Delete the directory and its contents
    shutil.rmtree(cache_dir)
    print(f"{cache_dir} has been deleted.")
else:
    print(f"{cache_dir} does not exist.")


/root/.cache has been deleted.


In [ ]:
from autogluon.core.metrics import make_scorer

ag_accuracy_scorer = make_scorer(name='accuracy',
                  score_func=sklearn.metrics.accuracy_score,
                  optimum=1,
                  greater_is_better=True,
                  needs_class=True)

In [ ]:
predictor.fit(..., calibrate_decision_threshold=True)

In [ ]:
for metric_name in scores:
    metric_score = scores[metric_name]
    metric_score_calibrated = scores_calibrated[metric_name]
    decision_threshold = predictor.decision_threshold
    print(f'decision_threshold={decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

In [ ]:
# loss 校准
predictor.set_decision_threshold(0.5)  # Reset decision threshold
for metric_name in ['f1', 'balanced_accuracy', 'mcc']:
    metric_score = predictor.evaluate(test_data, silent=True)[metric_name]
    calibrated_decision_threshold = predictor.calibrate_decision_threshold(metric=metric_name, verbose=False)
    metric_score_calibrated = predictor.evaluate(
        test_data, decision_threshold=calibrated_decision_threshold, silent=True
    )[metric_name]
    print(f'decision_threshold={calibrated_decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

In [ ]:
automl = TabularPredictor(label='Rings', problem_type='regression',
                          eval_metric=ag_RMSLE_scorer
                          )

automl.fit(train, presets='best_quality', time_limit=time_limit, num_bag_folds=8, num_bag_sets=0, num_stack_levels=1, dynamic_stacking=False,
            included_model_types=['XGB', 'CAT', 'GBM', 'XT'], ag_args_fit={'num_gpus': 1, 'num_cpus': 4},
          )

In [ ]:
from autogluon.common import space

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}  # Refer to TabularPredictor.fit docstring for all valid values

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

In [ ]:

hyperparameters = {
    'NN_TORCH': {'num_epochs': 10, 'activation': 'relu', 'dropout_prob': space.Real(0.0, 0.5)},
    'GBM': {'num_boost_round': 1000, 'learning_rate': space.Real(0.01, 0.1, log=True)},
    'XGB': {'n_estimators': 1000, 'learning_rate': space.Real(0.01, 0.1, log=True)}
}


In [ ]:
all_models = predictor.model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

## train

In [47]:
train = pd.read_csv('data/train.csv')

In [48]:
test = pd.read_csv('data/test.csv')

In [ ]:
train = pd.read_csv('openfe_train.csv')

In [ ]:
test = pd.read_csv('openfe_test.csv')

In [ ]:
# 划分训练集和测试集
train, val = train_test_split(train, random_state=22, test_size=0.1)

In [49]:
#time_limit = 3*60  # train various models for ~2 min
#num_trials = 10  # try at most 5 different hyperparameter configurations for each type of model
# recall precision 'f1'（二分类），'roc_auc'（二分类），'log_loss'（分类），'mean_absolute_error'（回归），'median_absolute_error'（回归）。
# precision balanced_accuracy
metric = 'balanced_accuracy' #
save_path = 'model'  # specifies folder to store trained models
# bayesopt  Hyperband
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler
# hyperparameter_tune_kwargs = {    'num_trials': 3,
#                   'scheduler' : 'local',
#                   'searcher': 'auto' #random', 'bayesopt' (Bayesian optimization) 'grid
# }
feature_prune_kwargs={"max_features_to_select":17 , "force_prune":True}


In [ ]:
X_val_transformed = pd.concat([X_train_transformed, y_train], axis=1)

In [50]:
#  eval_metric=metric,
#problem_type='regression',
predictor = TabularPredictor(label='Survived',
                eval_metric=metric,
                path=save_path,
                problem_type='binary').fit(
    train,
    #tuning_data=val,
    #num_gpus=1,
    ag_args_fit={'num_gpus': 1, 'num_cpus': 12},
    #time_limit=time_limit,
    num_bag_folds=5,
    #hyperparameters=hyperparameters,
    #hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    feature_prune_kwargs=feature_prune_kwargs,
    #num_bag_sets=1,
    num_stack_levels=1,
    #feature_generator=feature_generator,
    calibrate_decision_threshold=True,
    verbosity=2,
    dynamic_stacking=False,
    included_model_types=['XGB', 'CAT', 'GBM', 'NN_TORCH', 'RF'],
    presets='medium_quality' # medium_quality best_quality
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          12
Memory Avail:       47.64 GB / 52.96 GB (90.0%)
Disk Space Avail:   62.85 GB / 100.00 GB (62.8%)
Presets specified: ['medium_quality']
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
		Context path: "model/ds_sub_fit/sub_fit_ho"
Leaderboard on holdout data (DyStack):
                            model  score_holdout  score_val        eval_metric  pred_time_test  pred_time_val    fit_time  

## summery

|  f   |  s/e   | p
| :---   | :--: | :--: |
|0     |  0.892725/0.8130252100840335 | 0.78708|
|  35  |  0.900793/0.941701680672269  | 0.77751|
|  25  |  0.906272/0.8124999999999999 | 0.78229|
|  22   |0.847419  /0.7463235294117647| 0.80382|
|  19   |0.861420  /0.7316176470588236 | 0.79665|
|  17   |0.848028/0.7226890756302521|  0.80622|
|  17+   | 0.861420  /0.7316176470588236 | 0.77272|
|  15   |  0.850462  /0.7494747899159664   | 0.80143|
|  f   |  s/e   | p|
|  f   |  s/e   | p|
|  f   |  s/e   | p|

In [ ]:
|  f   |  s/e   | p
| :---   | :--: | :--: |
|0     |  0.892725/0.8130252100840335 | 0.78708|
|  35  |  0.900793/0.941701680672269  | 0.77751|
|  25  |  0.906272/0.8124999999999999 | 0.78229|
|  22   |0.847419  /0.7463235294117647| 0.80382|
|  19   |0.861420  /0.7316176470588236 | 0.79665|
|  17   |0.848028/0.7226890756302521|  0.80622|
|  17+   | 0.861420  | 0.77272|
|  17++ 1_t ds_F s2  | 0.841866      | 0.78468|
|  17+++ 1_t ds_T s1 |  0.824729     | p|
|  15   |  0.850462  /0.7494747899159664   | 0.80143|
|  15+ | 0.841866 | p|
|  f   |  s/e   | p|
|  f   |  s/e   | p|

In [ ]:
predictor.leaderboard(extra_info=True)

In [59]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                           model  score_val        eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0          LightGBM_BAG_L1_Prune   0.824729  balanced_accuracy       0.015143  19.756578                0.015143          19.756578            1       True          2
1            WeightedEnsemble_L2   0.824729  balanced_accuracy       0.015990  19.853772                0.000847           0.097194            2       True          9
2    NeuralNetTorch_BAG_L1_Prune   0.824537  balanced_accuracy       0.055044  31.358004                0.055044          31.358004            1       True          7
3           XGBoost_BAG_L1_Prune   0.823435  balanced_accuracy       0.069103  12.831349                0.069103          12.831349            1       True          6
4     LightGBMLarge_BAG_L1_Prune   0.822500  balanced_accuracy       0.011739  22.983251               

In [ ]:
predictor.evaluate(val)

NameError: name 'val' is not defined

In [ ]:
predictor.feature_importance(val)
# test_data

These features in provided data are not utilized by the predictor and will be ignored: ['Surname_8', 'Surname_12', 'Surname_36', 'Surname_44', 'Surname_52', 'Surname_59', 'Surname_85', 'Ticket_p_4', 'Ticket_p_10']
Computing feature importance via permutation shuffling for 180 features using 90 rows with 5 shuffle sets...
	797.03s	= Expected runtime (159.41s per shuffle set)
	10.69s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Age,0.051996,0.012046,0.000322,5,0.076798,0.027194
Pclass_2,0.048950,0.019127,0.002307,5,0.088331,0.009568
Title_2,0.039181,0.017661,0.003852,5,0.075546,0.002816
Cabin,0.033088,0.017690,0.006946,5,0.069513,-0.003336
Fare,0.020378,0.022616,0.057084,5,0.066944,-0.026188
...,...,...,...,...,...,...
Side,-0.005882,0.010234,0.865961,5,0.015190,-0.026955
Namesakes,-0.009454,0.006023,0.987663,5,0.002947,-0.021855
Title_3,-0.010084,0.008505,0.971550,5,0.007427,-0.027595
Embarked_1,-0.012920,0.003993,0.999032,5,-0.004699,-0.021142


## prune

In [ ]:
# Get the name of the best model
best_model_name = predictor.get_model_best()
print(f"Best model: {best_model_name}")

# Get the detailed info of the best model
model_info = predictor.info()  # Shows general predictor info
best_model_info = predictor.get_model_info(best_model_name)  # Best model's specific info
print(best_model_info)


In [ ]:
predictor.info()

In [ ]:
predictor.get_model_best()

'WeightedEnsemble_L3'

In [ ]:
all_models = predictor.model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

In [ ]:
all_models = predictor.model_names()
model_to_use = all_models[1]
specific_model = predictor._trainer.load_model(model_to_use)

Loading: model/models/LightGBM_BAG_L1/model.pkl


In [ ]:
specific_model.get_info()

In [ ]:
feature_prune_kwargs={"max_features_to_select": 30, "force_prune":True}

In [ ]:
hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}  # Refer to TabularPredictor.fit docstring for all valid values


## params

In [ ]:
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_cpus': 2, 'num_gpus': 1},
 'auto_stack': True,
 'num_bag_folds': 5,
 'num_bag_sets': 1,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_cpus': 2, 'num_gpus': 1},
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1111111111111111,
             'memory_safe_fits': True,
             'n_folds': 2,
             'n_repeats': 1,
             'validation_procedure': 'holdout'},
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'included_model_types': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 1,
 'num_stack_levels': None,
 'pseudo_data': None,
 'refit_full': False,
 'save_bag_folds': None,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}

In [ ]:

	'NN_TORCH'	'GBM'	'CAT' 'XGB'
	'FASTAI'	'RF'	'XT'	'KNN'

In [ ]:
{
	'NN_TORCH': {},
	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
	'CAT': {},
	'XGB': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
}

In [ ]:
min_data_in_leaf:   Int: lower=2, upper=60
num_leaves:   Int: lower=16, upper=96

## predict

In [ ]:
scores_calibrated = predictor.evaluate(X_val_transformed)

In [ ]:
ttn_test = feature_generator.transform(ttn_test)

In [52]:
pred = predictor.predict(test)

In [ ]:
pred

,Survived
0,0
1,0
2,0
3,0
4,0
...,...
413,0
414,1
415,0
416,0


## load model

In [ ]:
predictor = TabularPredictor.load("model")

## submit

In [ ]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
cd /content/drive/MyDrive/Kaggle/ttn

/content/drive/MyDrive/Kaggle/ttn


In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
# Install the kaggle API using
!pip install -q kaggle

In [ ]:
# Move the kaggle.json file into ~/.kaggle, which is where the API client expects your token to be located:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
gen = pd.read_csv('gender_submission.csv')

In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['Survived'] = pred
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)

In [53]:
submission = pd.DataFrame(columns=['PassengerId', 'Survived'])

In [54]:
submission['Survived'] = pred

In [55]:
submission['PassengerId'] = gen['PassengerId']

In [56]:
submission['Survived'] = submission['Survived'].astype(int)

In [57]:
submission.to_csv('submission.csv', index=False)

In [ ]:
!kaggle competitions list

:

In [58]:
!kaggle competitions submit -c titanic   -f submission.csv -m "submission"

100% 2.77k/2.77k [00:01<00:00, 1.61kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster